In [1]:
from itertools import combinations
import math
import os

from itertools import combinations
import lightgbm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process.kernels import *
from sklearn.gaussian_process import GaussianProcessRegressor

from rshdmrgpr.rs_hdmr_gpr1 import *

ModuleNotFoundError: No module named 'rshdmrgpr.rs_hdmr_gpr1'

In [ ]:
def print_kernel_info(models):
    idx = 1
    for model in models:
        print(f"Component function {idx} optimized kernel: {model.kernel_}")
        idx += 1

This notebook supplements the research paper:  
  
<font color='red'>**Random Sampling High Dimensional Model Representation Gaussian Process Regression (RS-HDMR-GPR): a code for representing multidimensional functions with lower-dimensional terms**</font>
    
The following section contains code used to generate the figures in Sections 3.2.

In [ ]:
# Extracts the data set
data = load_data('KED')

In [ ]:
data.min()

In [ ]:
# Scales the data set to be between [0, 1]
scale = data['out'].max() - data['out'].min()
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(data)
data_scaled = pd.DataFrame(scaler.transform(data), columns=data.columns)

In [ ]:
features = data_scaled.drop(columns=['out'])
labels = data_scaled['out']

In [ ]:
data.head()

In [ ]:
data.shape

##### Helper Functions

In [ ]:
def get_RMSE(y, y_pred):
    """Computes the RMSE between two vectors"""
    rmse = math.sqrt(mean_squared_error(y, y_pred))
    print(f'The RMSE is {rmse}')
    return rmse

### Full GPR Fits

In [ ]:
# We prepare 3 sets of training and testing data of sizes: 500, 2000 and 5000
x_train1, x_test1, y_train1, y_test1 = train_test_split(data_scaled.drop(columns=['out']), data_scaled['out'], train_size=500, test_size=None, random_state=42)
x_train2, x_test2, y_train2, y_test2 = train_test_split(data_scaled.drop(columns=['out']), data_scaled['out'], train_size=2000, test_size=None, random_state=42)
x_train3, x_test3, y_train3, y_test3 = train_test_split(data_scaled.drop(columns=['out']), data_scaled['out'], train_size=5000, test_size=None, random_state=42)
d = data.shape[1] - 1

In [ ]:
# FullGPR initializations. Only isotropic kernels are used.
gpr1 = GaussianProcessRegressor(kernel=RBF(0.5), alpha=5*1e-4, n_restarts_optimizer=1, optimizer=None)
gpr2 = GaussianProcessRegressor(kernel=RBF(0.5), alpha=5*1e-4, n_restarts_optimizer=1, optimizer=None)
gpr3 = GaussianProcessRegressor(kernel=RBF(0.5), alpha=5*1e-5, n_restarts_optimizer=1, optimizer=None)

In [ ]:
gpr1.fit(x_train1, y_train1)

In [ ]:
gpr2.fit(x_train2, y_train2)

In [ ]:
gpr3.fit(x_train3, y_train3)

In [ ]:
# Length scale results after fit:
print(gpr1.kernel_, gpr2.kernel_, gpr3.kernel_)

In [ ]:
y_pred1 = batch_predict(gpr1, features)

In [ ]:
y_pred2 = batch_predict(gpr2, features)

In [ ]:
y_pred3 = batch_predict(gpr3, features)

In [ ]:
get_RMSE(y_pred1 * scale, labels * scale)
get_RMSE(y_pred2 * scale, labels * scale)
get_RMSE(y_pred3 * scale, labels * scale)

In [ ]:
print('R^2 value for the 500 point fit is:', np.corrcoef(y_pred1 * scale, data_scaled['out'] * scale)[0, 1] ** 2)
print('R^2 value for the 2000 point fit is:', np.corrcoef(y_pred2 * scale, data_scaled['out'] * scale)[0, 1] ** 2)
print('R^2 value for the 5000 point fit is:', np.corrcoef(y_pred3 * scale, data_scaled['out'] * scale)[0, 1] ** 2)

In [ ]:
# 500 points
correlation_plot(data_scaled['out'] * scale, y_pred1 * scale, xlabel='Target', ylabel='Prediction')
# plot_for_paper(data_scaled['out'] * scale, y_pred1 * scale, xlabel='Target', ylabel='Prediction', name=f'fullGPR_500.png', save=True)

In [ ]:
# 2000 points
correlation_plot(data_scaled['out'] * scale, y_pred2 * scale, xlabel='Target', ylabel='Prediction')
# plot_for_paper(data_scaled['out'] * scale, y_pred2 * scale, xlabel='Target', ylabel='Prediction', name=f'fullGPR_2000.png', save=True)

In [ ]:
# 5000 points
correlation_plot(data_scaled['out'] * scale, y_pred3 * scale, xlabel='Target', ylabel='Prediction')
# plot_for_paper(data_scaled['out'] * scale, y_pred3 * scale, xlabel='Target', ylabel='Prediction', name=f'fullGPR_5000.png', save=True)

### HDMR FITS

In [ ]:
# Initializes the Model classes for training
matrices1, kernels1 = kernel_matrices(1, 7, length_scale=0.5)#, length_scale_bounds=(0.3, 10000))
matrices2, kernels2 = kernel_matrices(2, 7, length_scale=0.5)#, length_scale_bounds=(0.3, 10000))
matrices3, kernels3 = kernel_matrices(3, 7, length_scale=0.5)#, length_scale_bounds=(0.3, 10000))
matrices4, kernels4 = kernel_matrices(4, 7, length_scale=0.5)#, length_scale_bounds=(0.3, 10000))
matrices5, kernels5 = kernel_matrices(5, 7, length_scale=0.5)#, length_scale_bounds=(0.3, 10000))
matrices6, kernels6 = kernel_matrices(6, 7, length_scale=0.5)#, length_scale_bounds=(0.3, 10000))
matrices7, kernels7 = kernel_matrices(7, 7, length_scale=0.5)#, length_scale_bounds=(0.3, 10000))
hdmr_1d = RSHDMRGPR(matrices1, kernels1)
hdmr_2d = RSHDMRGPR(matrices2, kernels2)
hdmr_3d = RSHDMRGPR(matrices3, kernels3)
hdmr_4d = RSHDMRGPR(matrices4, kernels4)
hdmr_5d = RSHDMRGPR(matrices5, kernels5)
hdmr_6d = RSHDMRGPR(matrices6, kernels6)
hdmr_7d = RSHDMRGPR(matrices7, kernels7)

In [ ]:
# models = [hdmr_1d, hdmr_2d, hdmr_3d, hdmr_4d, hdmr_5d,hdmr_6d, hdmr_7d]
hdmr = [hdmr_1d, hdmr_2d, hdmr_3d, hdmr_4d, hdmr_5d, hdmr_6d, hdmr_7d]
alphas = [3 * 1e-3, 8 * 1e-4, 3 * 1e-4, 8 * 1e-5, 3 * 1e-5, 8 * 1e-6, 3 * 1e-6]

In [ ]:
sequential_fitting(x_train1, y_train1, hdmr, alphas=alphas, cycles=50, optimizer="fmin_l_bfgs_b", opt_every=5, scale_down=(0.2, 2))

In [ ]:
preds1 = sequential_prediction(data_scaled.drop(columns=['out']), hdmr)

In [ ]:
i = 1
for p in preds1:
    v = np.corrcoef(p * scale, data_scaled['out'] * scale)[0,1]
    print(f'R^2 for the {i}d-hdmr fit is', v ** 2)
    i += 1

In [ ]:
i = 1
for p in preds1:
    get_RMSE(p * scale, data_scaled['out'] * scale)
    i += 1

In [ ]:
idx = 1
for model in hdmr:
    print(f"Model {idx}'s hyperparameters:")
    print_kernel_info(model.get_models())
    idx += 1